<a href="https://colab.research.google.com/github/wstiq/123456/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_notebook53fd1082eb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import os ; os.environ['OMP_NUM_THREADS'] = '4'

# Any results you write to the current directory are saved as output.

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np
import operator
import random
import datetime


from lightgbm import LGBMRegressor
import sklearn.discriminant_analysis
import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import hamming_loss, accuracy_score 
from pandas import DataFrame
from datetime import datetime
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from google.colab import drive

%matplotlib inline
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [79]:
#importing two csv files to work with/с помощью пандас считаем два цсв файла, с которыми нам предстоит работать
drive.mount('/content/drive')
datatrain = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Проект/train.csv')
datatest = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Проект/test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
datatrain.head()
#здесь и далее функции .head() и . tail() будут использоваться не столько для вывода информации, сколько для сохранения прогресса
#hereinafter functions .head() and .tail() are supposed to be used in purpose of saving progress

,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue
0,1,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-23,ConfirmedCases,0
1,2,NaN,NaN,Afghanistan,27657145,0.583587,2020-01-23,Fatalities,0
2,3,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-24,ConfirmedCases,0
3,4,NaN,NaN,Afghanistan,27657145,0.583587,2020-01-24,Fatalities,0
4,5,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-25,ConfirmedCases,0


In [6]:
datatrain.describe()
#description of our train dataset; we're interested in existence needed variables (969640)
#описание датасета, нас интересует то, что все необходимые значения присутствуют (969640)

,Id,Population,Weight,TargetValue
count,969640.000000,9.696400e+05,969640.000000,969640.000000
mean,484820.500000,2.720127e+06,0.530870,12.563518
std,279911.101847,3.477771e+07,0.451909,302.524795
min,1.000000,8.600000e+01,0.047491,-10034.000000
25%,242410.750000,1.213300e+04,0.096838,0.000000
50%,484820.500000,3.053100e+04,0.349413,0.000000
75%,727230.250000,1.056120e+05,0.968379,0.000000
max,969640.000000,1.395773e+09,2.239186,36163.000000


In [7]:
datatrain["TargetValue"] = datatrain["TargetValue"].abs()
#(about code above. when we have tried to work with dataset for the first time, we have discovered, that some variables of Confirmed cases are negative.
#We have been searching for reason why is this possible and desided to use .abs() function, because in dataset were parsed incorrect data)
#когда мы только начали работу с проектом, мы обнаружили, что присутствует 3800+ отрицательных значений у переменной с количеством заболевших
#было решено исправить все негативные значени на их абсолютную величину по модулю, поскольку у нас по официальным данным картина складывалась такой, как
#сообщалось в официальных источниках. PS: Артур посчитал, что тире парсилось вместо дефиса по пробелу, из-за этого кастилось в инт неправильно.
datatrain.info()
datatrain.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969640 entries, 0 to 969639
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Id              969640 non-null  int64  
 1   County          880040 non-null  object 
 2   Province_State  917280 non-null  object 
 3   Country_Region  969640 non-null  object 
 4   Population      969640 non-null  int64  
 5   Weight          969640 non-null  float64
 6   Date            969640 non-null  object 
 7   Target          969640 non-null  object 
 8   TargetValue     969640 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 66.6+ MB


,Id,Population,Weight,TargetValue
count,969640.000000,9.696400e+05,969640.000000,969640.000000
mean,484820.500000,2.720127e+06,0.530870,12.646364
std,279911.101847,3.477771e+07,0.451909,302.521343
min,1.000000,8.600000e+01,0.047491,0.000000
25%,242410.750000,1.213300e+04,0.096838,0.000000
50%,484820.500000,3.053100e+04,0.349413,0.000000
75%,727230.250000,1.056120e+05,0.968379,0.000000
max,969640.000000,1.395773e+09,2.239186,36163.000000


In [8]:
date = pd.to_datetime(datatrain["Date"])
datet = pd.to_datetime(datatest["Date"])
print (date)
datatrain["Date"] = date
datatest["Date"] = datet

0        2020-01-23
1        2020-01-23
2        2020-01-24
3        2020-01-24
4        2020-01-25
            ...    
969635   2020-06-08
969636   2020-06-09
969637   2020-06-09
969638   2020-06-10
969639   2020-06-10
Name: Date, Length: 969640, dtype: datetime64[ns]


In [9]:
datatrain["Date"]

0        2020-01-23
1        2020-01-23
2        2020-01-24
3        2020-01-24
4        2020-01-25
            ...    
969635   2020-06-08
969636   2020-06-09
969637   2020-06-09
969638   2020-06-10
969639   2020-06-10
Name: Date, Length: 969640, dtype: datetime64[ns]

In [10]:
ldate = int(len(date))
ldatet = int(len(datet))
print("Length of training- date is", ldate)
print("Length of test- date is", ldatet)

Length of training- date is 969640
Length of test- date is 311670


In [11]:
m = []
d = []
for i in range(0,ldate):
    dx = (date[i].strftime("%d"))
    mx = (date[i].strftime("%m"))
    m.append(int(mx))
    d.append(int(dx))

mt = []
dt = []
for i in range(0,ldatet):
    dtx = (datet[i].strftime("%d"))
    mtx = (datet[i].strftime("%m"))
    mt.append(int(mtx))
    dt.append(int(dtx))
#here we are trying to fill variables m, d, mt and dt in purpose of having dataset` dates representation
#здесь мы создаем переменные для того, чтобы получить представление о временных рамках датасета


In [12]:
train = datatrain
test = datatest


## Added Input of Month + Day from Date

In [13]:
train.insert(6,"Month",m,False)
train.insert(7,"Day",d,False)
test.insert(4,"Month",mt,False)
test.insert(5,"Day",dt,False)
#inserting day and month, will try to group up and work with concrete date if needed
#вставим переменные день и месяц для более удобной работы, в случае необходимости сгруппируем и отправим

In [14]:
train.head()

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue
0,1,NaN,NaN,Afghanistan,27657145,0.058359,1,23,2020-01-23,ConfirmedCases,0
1,2,NaN,NaN,Afghanistan,27657145,0.583587,1,23,2020-01-23,Fatalities,0
2,3,NaN,NaN,Afghanistan,27657145,0.058359,1,24,2020-01-24,ConfirmedCases,0
3,4,NaN,NaN,Afghanistan,27657145,0.583587,1,24,2020-01-24,Fatalities,0
4,5,NaN,NaN,Afghanistan,27657145,0.058359,1,25,2020-01-25,ConfirmedCases,0


In [15]:
train.tail(20)

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue
969620,969621,NaN,NaN,Zimbabwe,14240168,0.060711,6,1,2020-06-01,ConfirmedCases,25
969621,969622,NaN,NaN,Zimbabwe,14240168,0.607106,6,1,2020-06-01,Fatalities,0
969622,969623,NaN,NaN,Zimbabwe,14240168,0.060711,6,2,2020-06-02,ConfirmedCases,3
969623,969624,NaN,NaN,Zimbabwe,14240168,0.607106,6,2,2020-06-02,Fatalities,0
969624,969625,NaN,NaN,Zimbabwe,14240168,0.060711,6,3,2020-06-03,ConfirmedCases,16
969625,969626,NaN,NaN,Zimbabwe,14240168,0.607106,6,3,2020-06-03,Fatalities,0
969626,969627,NaN,NaN,Zimbabwe,14240168,0.060711,6,4,2020-06-04,ConfirmedCases,15
969627,969628,NaN,NaN,Zimbabwe,14240168,0.607106,6,4,2020-06-04,Fatalities,0
969628,969629,NaN,NaN,Zimbabwe,14240168,0.060711,6,5,2020-06-05,ConfirmedCases,28
969629,969630,NaN,NaN,Zimbabwe,14240168,0.607106,6,5,2020-06-05,Fatalities,0


In [16]:
test.head()

,ForecastId,County,Province_State,Country_Region,Month,Day,Population,Weight,Date,Target
0,1,NaN,NaN,Afghanistan,4,27,27657145,0.058359,2020-04-27,ConfirmedCases
1,2,NaN,NaN,Afghanistan,4,27,27657145,0.583587,2020-04-27,Fatalities
2,3,NaN,NaN,Afghanistan,4,28,27657145,0.058359,2020-04-28,ConfirmedCases
3,4,NaN,NaN,Afghanistan,4,28,27657145,0.583587,2020-04-28,Fatalities
4,5,NaN,NaN,Afghanistan,4,29,27657145,0.058359,2020-04-29,ConfirmedCases


In [17]:
print("Datatrain -------------------------------------")
traindays = datatrain['Date'].nunique()
print("Number of Country_Region: ", datatrain['Country_Region'].nunique())
print("Number of Province_State: ", datatrain['Province_State'].nunique())
print("Number of Days: ", traindays)

notrain = datatrain['Id'].nunique()
print("Number of datapoints in train:", notrain)
lotrain = int(notrain/traindays)
print("L Trains:", lotrain)
#187 countries and 140 days
#187 стран и 140 дней

Datatrain -------------------------------------
Number of Country_Region:  187
Number of Province_State:  133
Number of Days:  140
Number of datapoints in train: 969640
L Trains: 6926


In [18]:
print("Datatest -------------------------------------")
testdays = datatest['Date'].nunique()
print("Number of Country_Region: ", datatest['Country_Region'].nunique())
print("Number of Province_State: ", datatest['Province_State'].nunique())
print("Number of Days: ", testdays)
notest = datatest['ForecastId'].nunique()
print("Number of datapoints in test:", notest)
lotest = int(notest/testdays)
print("L Test:", lotest)


Datatest -------------------------------------
Number of Country_Region:  187
Number of Province_State:  133
Number of Days:  45
Number of datapoints in test: 311670
L Test: 6926


In [19]:
train['Target2'] = train.Target.factorize()[0]
test['Target2'] = test.Target.factorize()[0]

In [20]:
train.head()

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue,Target2
0,1,NaN,NaN,Afghanistan,27657145,0.058359,1,23,2020-01-23,ConfirmedCases,0,0
1,2,NaN,NaN,Afghanistan,27657145,0.583587,1,23,2020-01-23,Fatalities,0,1
2,3,NaN,NaN,Afghanistan,27657145,0.058359,1,24,2020-01-24,ConfirmedCases,0,0
3,4,NaN,NaN,Afghanistan,27657145,0.583587,1,24,2020-01-24,Fatalities,0,1
4,5,NaN,NaN,Afghanistan,27657145,0.058359,1,25,2020-01-25,ConfirmedCases,0,0


In [21]:
test.head()

,ForecastId,County,Province_State,Country_Region,Month,Day,Population,Weight,Date,Target,Target2
0,1,NaN,NaN,Afghanistan,4,27,27657145,0.058359,2020-04-27,ConfirmedCases,0
1,2,NaN,NaN,Afghanistan,4,27,27657145,0.583587,2020-04-27,Fatalities,1
2,3,NaN,NaN,Afghanistan,4,28,27657145,0.058359,2020-04-28,ConfirmedCases,0
3,4,NaN,NaN,Afghanistan,4,28,27657145,0.583587,2020-04-28,Fatalities,1
4,5,NaN,NaN,Afghanistan,4,29,27657145,0.058359,2020-04-29,ConfirmedCases,0


In [22]:
traincount = int(len(train["Date"]))
testcount = int(len(test["Date"]))

In [80]:
train.Province_State = train.Province_State.fillna(0)
train.County = train.County.fillna(0)
empty = 0
for i in range(0,traincount):
    if(train.Province_State[i] == empty):
        train.Province_State[i] = train.Country_Region[i];
   #here and below we do have to fix one very important thing. if country has no counties or states, then we have to duplicate variable into two NaNs, cause of
   #groupping up and working after. Otherwise, we will get wrong country stat, and we wont be able to make our model good
   #!!Проблема, о которой не забыть рассказать
   #!!Спросить как можно делать это не 10 минут
   #Заполним отсутствующие переменные значениями страны, чтобы не упустить никаких строк в нашей модели
   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
for i in range(0,traincount):
    if(train.County[i] == empty):
        train.County[i] = train.Province_State[i];

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
train.head()

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue,Target2
0,1,Afghanistan,Afghanistan,Afghanistan,27657145,0.058359,1,23,2020-01-23,ConfirmedCases,0,0
1,2,Afghanistan,Afghanistan,Afghanistan,27657145,0.583587,1,23,2020-01-23,Fatalities,0,1
2,3,Afghanistan,Afghanistan,Afghanistan,27657145,0.058359,1,24,2020-01-24,ConfirmedCases,0,0
3,4,Afghanistan,Afghanistan,Afghanistan,27657145,0.583587,1,24,2020-01-24,Fatalities,0,1
4,5,Afghanistan,Afghanistan,Afghanistan,27657145,0.058359,1,25,2020-01-25,ConfirmedCases,0,0


In [26]:
train["Pop"] = np.log1p(train["Population"])
test["Pop"] = np.log1p(test["Population"])
#as always, we got to use log1p to make population variable SLIGHLY LESS THAN ITS ABSOLUTE VALUE
#воспользуемся стандартным приёмом, уменьшим значение переменной население, прологарифмировав её.

In [27]:
train.head()

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue,Target2,Pop
0,1,Afghanistan,Afghanistan,Afghanistan,27657145,0.058359,1,23,2020-01-23,ConfirmedCases,0,0,17.135395
1,2,Afghanistan,Afghanistan,Afghanistan,27657145,0.583587,1,23,2020-01-23,Fatalities,0,1,17.135395
2,3,Afghanistan,Afghanistan,Afghanistan,27657145,0.058359,1,24,2020-01-24,ConfirmedCases,0,0,17.135395
3,4,Afghanistan,Afghanistan,Afghanistan,27657145,0.583587,1,24,2020-01-24,Fatalities,0,1,17.135395
4,5,Afghanistan,Afghanistan,Afghanistan,27657145,0.058359,1,25,2020-01-25,ConfirmedCases,0,0,17.135395


In [28]:
test.Province_State = test.Province_State.fillna(0)
test.County = test.County.fillna(0)
empty = 0
for i in range(0,testcount):
    if(test.Province_State[i] == empty):
        test.Province_State[i] = test.Country_Region[i];


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
empty=0
for i in range(0,testcount):    
    if(test.County[i] == 0):
        test.County[i] = test.Province_State[i];

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
test.head()

,ForecastId,County,Province_State,Country_Region,Month,Day,Population,Weight,Date,Target,Target2,Pop
0,1,Afghanistan,Afghanistan,Afghanistan,4,27,27657145,0.058359,2020-04-27,ConfirmedCases,0,17.135395
1,2,Afghanistan,Afghanistan,Afghanistan,4,27,27657145,0.583587,2020-04-27,Fatalities,1,17.135395
2,3,Afghanistan,Afghanistan,Afghanistan,4,28,27657145,0.058359,2020-04-28,ConfirmedCases,0,17.135395
3,4,Afghanistan,Afghanistan,Afghanistan,4,28,27657145,0.583587,2020-04-28,Fatalities,1,17.135395
4,5,Afghanistan,Afghanistan,Afghanistan,4,29,27657145,0.058359,2020-04-29,ConfirmedCases,0,17.135395


In [31]:
label = preprocessing.LabelEncoder()
train.Country_Region = label.fit_transform(train.Country_Region)
train.Province_State = label.fit_transform(train.Province_State)
train.County = label.fit_transform(train.County)
train.head()
#using LabelEncoder()
#использование ЛейблЭнкодера

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue,Target2,Pop
0,1,7,0,0,27657145,0.058359,1,23,2020-01-23,ConfirmedCases,0,0,17.135395
1,2,7,0,0,27657145,0.583587,1,23,2020-01-23,Fatalities,0,1,17.135395
2,3,7,0,0,27657145,0.058359,1,24,2020-01-24,ConfirmedCases,0,0,17.135395
3,4,7,0,0,27657145,0.583587,1,24,2020-01-24,Fatalities,0,1,17.135395
4,5,7,0,0,27657145,0.058359,1,25,2020-01-25,ConfirmedCases,0,0,17.135395


In [32]:
test.Country_Region = label.fit_transform(test.Country_Region)
test.Province_State = label.fit_transform(test.Province_State)
test.County = label.fit_transform(test.County)
test.head()

,ForecastId,County,Province_State,Country_Region,Month,Day,Population,Weight,Date,Target,Target2,Pop
0,1,7,0,0,4,27,27657145,0.058359,2020-04-27,ConfirmedCases,0,17.135395
1,2,7,0,0,4,27,27657145,0.583587,2020-04-27,Fatalities,1,17.135395
2,3,7,0,0,4,28,27657145,0.058359,2020-04-28,ConfirmedCases,0,17.135395
3,4,7,0,0,4,28,27657145,0.583587,2020-04-28,Fatalities,1,17.135395
4,5,7,0,0,4,29,27657145,0.058359,2020-04-29,ConfirmedCases,0,17.135395


In [33]:
c_train = train.loc[train['Target'] == 'ConfirmedCases']
f_train = train.loc[train['Target'] == 'Fatalities']

c_test = test.loc[test['Target'] == 'ConfirmedCases']
f_test = test.loc[test['Target'] == 'Fatalities']

td = datet[0]
tdate = c_train.loc[c_train['Date'] == td]
c_train.head(90)

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue,Target2,Pop
0,1,7,0,0,27657145,0.058359,1,23,2020-01-23,ConfirmedCases,0,0,17.135395
2,3,7,0,0,27657145,0.058359,1,24,2020-01-24,ConfirmedCases,0,0,17.135395
4,5,7,0,0,27657145,0.058359,1,25,2020-01-25,ConfirmedCases,0,0,17.135395
6,7,7,0,0,27657145,0.058359,1,26,2020-01-26,ConfirmedCases,0,0,17.135395
8,9,7,0,0,27657145,0.058359,1,27,2020-01-27,ConfirmedCases,0,0,17.135395
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,171,7,0,0,27657145,0.058359,4,17,2020-04-17,ConfirmedCases,66,0,17.135395
172,173,7,0,0,27657145,0.058359,4,18,2020-04-18,ConfirmedCases,27,0,17.135395
174,175,7,0,0,27657145,0.058359,4,19,2020-04-19,ConfirmedCases,63,0,17.135395
176,177,7,0,0,27657145,0.058359,4,20,2020-04-20,ConfirmedCases,30,0,17.135395


In [34]:
datet

0        2020-04-27
1        2020-04-27
2        2020-04-28
3        2020-04-28
4        2020-04-29
            ...    
311665   2020-06-08
311666   2020-06-09
311667   2020-06-09
311668   2020-06-10
311669   2020-06-10
Name: Date, Length: 311670, dtype: datetime64[ns]

In [35]:
f_train.tail(10)

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue,Target2,Pop
969621,969622,2135,318,186,14240168,0.607106,6,1,2020-06-01,Fatalities,0,1,16.471577
969623,969624,2135,318,186,14240168,0.607106,6,2,2020-06-02,Fatalities,0,1,16.471577
969625,969626,2135,318,186,14240168,0.607106,6,3,2020-06-03,Fatalities,0,1,16.471577
969627,969628,2135,318,186,14240168,0.607106,6,4,2020-06-04,Fatalities,0,1,16.471577
969629,969630,2135,318,186,14240168,0.607106,6,5,2020-06-05,Fatalities,0,1,16.471577
969631,969632,2135,318,186,14240168,0.607106,6,6,2020-06-06,Fatalities,0,1,16.471577
969633,969634,2135,318,186,14240168,0.607106,6,7,2020-06-07,Fatalities,0,1,16.471577
969635,969636,2135,318,186,14240168,0.607106,6,8,2020-06-08,Fatalities,0,1,16.471577
969637,969638,2135,318,186,14240168,0.607106,6,9,2020-06-09,Fatalities,0,1,16.471577
969639,969640,2135,318,186,14240168,0.607106,6,10,2020-06-10,Fatalities,0,1,16.471577


In [36]:
zt = datet[0]
daycount = []
lt = int(lotrain/2) 
for i in range(0,lt):
    for j in range(1,traindays+1):
        daycount.append(j)
#Since the dataset is divided into confirmed and fatalities, lotrain is divided by 2 since there are 2 targets.
#Учитывая, что датасет имеет две переменные, а именно заболевания и смерти, разделим на два.


In [37]:
len(daycount)

484820

In [38]:
tx = traindays * 2
zo = []

for i in range(0,tx):
    if(datet[0] == date[i]):
        zx = i
        zo.append(zx)


In [39]:
print(zo)

[190, 191]


In [40]:
zx = zo[0]/2
print(zx)

95.0


In [41]:
daytest = []
lt2 = int(lotest/2)
for i in range(0,lt2):
    for j in range(1,testdays+1):
        jr = zx + j
        daytest.append(jr)


In [42]:
len(daytest)

155835

In [43]:
daytest

[96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0,
 139.0,
 140.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0,
 139.0,
 140.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132

In [44]:
c_train.head()

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue,Target2,Pop
0,1,7,0,0,27657145,0.058359,1,23,2020-01-23,ConfirmedCases,0,0,17.135395
2,3,7,0,0,27657145,0.058359,1,24,2020-01-24,ConfirmedCases,0,0,17.135395
4,5,7,0,0,27657145,0.058359,1,25,2020-01-25,ConfirmedCases,0,0,17.135395
6,7,7,0,0,27657145,0.058359,1,26,2020-01-26,ConfirmedCases,0,0,17.135395
8,9,7,0,0,27657145,0.058359,1,27,2020-01-27,ConfirmedCases,0,0,17.135395


In [45]:
f_train.head()

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,Date,Target,TargetValue,Target2,Pop
1,2,7,0,0,27657145,0.583587,1,23,2020-01-23,Fatalities,0,1,17.135395
3,4,7,0,0,27657145,0.583587,1,24,2020-01-24,Fatalities,0,1,17.135395
5,6,7,0,0,27657145,0.583587,1,25,2020-01-25,Fatalities,0,1,17.135395
7,8,7,0,0,27657145,0.583587,1,26,2020-01-26,Fatalities,0,1,17.135395
9,10,7,0,0,27657145,0.583587,1,27,2020-01-27,Fatalities,0,1,17.135395


In [46]:
print("Length of Date in train:",len(c_train["Date"]))
print("Length of Daycount:",len(daycount))

Length of Date in train: 484820
Length of Daycount: 484820


## Added DayCount as Input from Dates.

In [47]:
c_train.insert(8,"DayCount",daycount,False)
c_test.insert(6,"DayCount",daytest,False)
#Hereinafter, we do start using variable DayCount (Jan 22 - 1st day) to have a representation of pandemic flow
#Здесь и далее, имеет смысл использовани переменной DayCount, посколько только таким образом
#мы можем воспользоваться представлением о течении пандемии. Нас не интересует конкретная дата, мы работаем на определенном промежутке.
#Более того, зная день начала эпидемии (22 января), мы можем спокойно восстановить дату

In [48]:
c_test.head()

,ForecastId,County,Province_State,Country_Region,Month,Day,DayCount,Population,Weight,Date,Target,Target2,Pop
0,1,7,0,0,4,27,96.0,27657145,0.058359,2020-04-27,ConfirmedCases,0,17.135395
2,3,7,0,0,4,28,97.0,27657145,0.058359,2020-04-28,ConfirmedCases,0,17.135395
4,5,7,0,0,4,29,98.0,27657145,0.058359,2020-04-29,ConfirmedCases,0,17.135395
6,7,7,0,0,4,30,99.0,27657145,0.058359,2020-04-30,ConfirmedCases,0,17.135395
8,9,7,0,0,5,1,100.0,27657145,0.058359,2020-05-01,ConfirmedCases,0,17.135395


In [49]:
f_train.insert(8,"DayCount",daycount,False)

In [50]:
f_test.insert(6,"DayCount",daytest,False)
f_train.head()

,Id,County,Province_State,Country_Region,Population,Weight,Month,Day,DayCount,Date,Target,TargetValue,Target2,Pop
1,2,7,0,0,27657145,0.583587,1,23,1,2020-01-23,Fatalities,0,1,17.135395
3,4,7,0,0,27657145,0.583587,1,24,2,2020-01-24,Fatalities,0,1,17.135395
5,6,7,0,0,27657145,0.583587,1,25,3,2020-01-25,Fatalities,0,1,17.135395
7,8,7,0,0,27657145,0.583587,1,26,4,2020-01-26,Fatalities,0,1,17.135395
9,10,7,0,0,27657145,0.583587,1,27,5,2020-01-27,Fatalities,0,1,17.135395


In [51]:
f_test.head()

,ForecastId,County,Province_State,Country_Region,Month,Day,DayCount,Population,Weight,Date,Target,Target2,Pop
1,2,7,0,0,4,27,96.0,27657145,0.583587,2020-04-27,Fatalities,1,17.135395
3,4,7,0,0,4,28,97.0,27657145,0.583587,2020-04-28,Fatalities,1,17.135395
5,6,7,0,0,4,29,98.0,27657145,0.583587,2020-04-29,Fatalities,1,17.135395
7,8,7,0,0,4,30,99.0,27657145,0.583587,2020-04-30,Fatalities,1,17.135395
9,10,7,0,0,5,1,100.0,27657145,0.583587,2020-05-01,Fatalities,1,17.135395


In [52]:
cX = np.c_[c_train["County"], c_train["Province_State"], c_train["Country_Region"], c_train["DayCount"],c_train["Pop"], c_train["Weight"]]
fX = np.c_[f_train["County"], f_train["Province_State"], f_train["Country_Region"], f_train["DayCount"],f_train["Pop"], f_train["Weight"]]

cXt = np.c_[c_test["County"], c_test["Province_State"], c_test["Country_Region"], c_test["DayCount"], c_test["Pop"],c_test["Weight"]]
fXt = np.c_[f_test["County"], f_test["Province_State"], f_test["Country_Region"], f_test["DayCount"], f_test["Pop"], f_test["Weight"]]
#here we choose variables to train
#выбираем переменные для использования в датасетах

In [53]:
cy = c_train["TargetValue"]
fy = f_train["TargetValue"]

In [54]:
regr = XGBRegressor(n_estimators = 600 ,objective='reg:squarederror', alpha = 0, gamma = 0, learning_rate = 0.06,  random_state = 42 , max_depth = 15, nthread = 4) 
regr1 = XGBRegressor(n_estimators = 600 ,objective='reg:squarederror', alpha = 0, gamma = 0, learning_rate = 0.06,  random_state = 42 , max_depth = 55, nthread = 4 ) 

In [81]:
regr.fit(cX,cy.ravel())
yscore = regr.predict(cX)
yscore = np.round(yscore)
ascore = mean_squared_error(yscore,cy)
print("Training (ConfirmedCases) - Mean Squared Error is: ",ascore)
#!!Здесь стоит упомянуть почему мы воспользовались именно градиентным бустингом и то, к какому оптимальному значению мы пришли, чтобы не переобучить модель
#Попытки использовать линейную регрессию проваливаются, это объясняется очень просто. У нас не существует условно какой-то прямой y=kx+b, вокруг которой мы можем провести
#исследование, то есть мы не можем адекватно обучить модель. Соответственно из популярных алгоритвом остаётся random forests, AdaBoost, LightGBM, XGBoost or CatBoost.
#исследование с использованием random forests показывает неплохие результаты, однако наш выбор был между AdaBoost и XGBoost
#Угадаете почему XGBoost? Потому что нам не принципиальна визуализация и важна скорость на датасете в миллион строк.
#Long story short:
#Compared to random forests and XGBoost, AdaBoost performs worse when irrelevant features are included in the model
#as shown by my time series analysis of bike sharing demand. Moreover, AdaBoost is not optimized for speed, therefore being significantly slower than XGBoost.

Training (ConfirmedCases) - Mean Squared Error is:  10.646726620188936


In [56]:
regr1.fit(fX,fy.ravel())
yscore = regr1.predict(fX)
yscore = np.round(yscore)
ascore = mean_squared_error(yscore,fy)
print("Training (Fatalities) - Mean Squared Error is: ",ascore)

Training (Fatalities) - Mean Squared Error is:  0.0


In [57]:
#regr = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.04,  random_state = 42 , max_depth = 23) #Training (ConfirmedCases) - Mean Squared Error is:  0.0005750290495952613
#regr1 = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.04,  random_state = 42 , max_depth = 23)

#regr = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23) # Best
#regr1 = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23) 

In [58]:
#regr = XGBRegressor(n_estimators = 2250 ,min_child_weight = 1, gamma = 1.15, learning_rate = 0.035,  random_state = 42 , max_depth = 23) # 0.0055
#regr1 = XGBRegressor(n_estimators = 2250 ,min_child_weight = 1, gamma = 1.15, learning_rate = 0.035,  random_state = 42 , max_depth = 23) 

In [59]:
#regr = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23) # Best
#regr1 = XGBRegressor(n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23)
#despite of getting best parameters above, we cant allow ourselves to use it, because of limited sources of our PCs and internet. If you have enough time - uncomment and check <3
#Несмотря на то, что наилучшим образом себя показали параметры из первой строки, модель обучается очень долго. Поэтому в рамках обучающего проекта - это простительно.

In [60]:
ypred = regr.predict(cXt)
ypred = np.round(ypred)
ypred = pd.DataFrame({'ConfirmedCases' : ypred}) 
ypred.head(20) 

,ConfirmedCases
0,161.0
1,127.0
2,122.0
3,223.0
4,164.0
5,147.0
6,221.0
7,201.0
8,315.0
9,176.0


In [61]:
ypred.tail(20)

,ConfirmedCases
155815,6.0
155816,3.0
155817,-0.0
155818,-3.0
155819,-0.0
155820,79.0
155821,16.0
155822,3.0
155823,20.0
155824,5.0


In [62]:
ypred2 = regr1.predict(fXt)
ypred2 = np.round(ypred2)

In [63]:
ypred2 = pd.DataFrame({'Fatalities' : ypred2}) 
ypred2.head(20) 

,Fatalities
0,7.0
1,1.0
2,2.0
3,4.0
4,4.0
5,4.0
6,13.0
7,5.0
8,5.0
9,9.0


In [64]:
ypred2.tail(20)

,Fatalities
155815,-0.0
155816,-0.0
155817,-0.0
155818,-0.0
155819,0.0
155820,-0.0
155821,0.0
155822,0.0
155823,0.0
155824,0.0


In [65]:
len(ypred)

155835

In [66]:
ypc = pd.DataFrame()
forecast = c_test["ForecastId"]
ypc.insert(0,"ForecastId",forecast,False)
ypc.reset_index(drop=True, inplace=True)
ypc.insert(1,"TargetValue",ypred,False)
ypc.head()

,ForecastId,TargetValue
0,1,161.0
1,3,127.0
2,5,122.0
3,7,223.0
4,9,164.0


In [67]:
len(ypc)

155835

In [68]:
ypc.tail()

,ForecastId,TargetValue
155830,311661,15.0
155831,311663,3.0
155832,311665,9.0
155833,311667,25.0
155834,311669,6.0


In [69]:
forecast2 = f_test["ForecastId"]
ypf = pd.DataFrame()
ypf.insert(0,"ForecastId",forecast2,False)
ypf.reset_index(drop=True, inplace=True)
ypf.insert(1,"TargetValue",ypred2,False)
ypf.head()

,ForecastId,TargetValue
0,2,7.0
1,4,1.0
2,6,2.0
3,8,4.0
4,10,4.0


In [70]:
len(ypf)

155835

In [71]:
ypf.tail()

,ForecastId,TargetValue
155830,311662,0.0
155831,311664,0.0
155832,311666,0.0
155833,311668,-0.0
155834,311670,-0.0


In [72]:
a=ypc.groupby(['ForecastId'])['TargetValue'].quantile(q=0.05).reset_index()
b=ypc.groupby(['ForecastId'])['TargetValue'].quantile(q=0.5).reset_index()
c=ypc.groupby(['ForecastId'])['TargetValue'].quantile(q=0.95).reset_index()

a.columns=['Id','q0.05']
b.columns=['Id','q0.5']
c.columns=['Id','q0.95']
a=pd.concat([a,b['q0.5'],c['q0.95']],1)
a
#приводим к необходимому виду
#here we are making for required view

,Id,q0.05,q0.5,q0.95
0,1,161.0,161.0,161.0
1,3,127.0,127.0,127.0
2,5,122.0,122.0,122.0
3,7,223.0,223.0,223.0
4,9,164.0,164.0,164.0
...,...,...,...,...
155830,311661,15.0,15.0,15.0
155831,311663,3.0,3.0,3.0
155832,311665,9.0,9.0,9.0
155833,311667,25.0,25.0,25.0


In [73]:
a2=ypf.groupby(['ForecastId'])['TargetValue'].quantile(q=0.05).reset_index()
b2=ypf.groupby(['ForecastId'])['TargetValue'].quantile(q=0.5).reset_index()
c2=ypf.groupby(['ForecastId'])['TargetValue'].quantile(q=0.95).reset_index()

a2.columns=['Id','q0.05']
b2.columns=['Id','q0.5']
c2.columns=['Id','q0.95']
a2=pd.concat([a2,b2['q0.5'],c2['q0.95']],1)
a2

,Id,q0.05,q0.5,q0.95
0,2,7.0,7.0,7.0
1,4,1.0,1.0,1.0
2,6,2.0,2.0,2.0
3,8,4.0,4.0,4.0
4,10,4.0,4.0,4.0
...,...,...,...,...
155830,311662,0.0,0.0,0.0
155831,311664,0.0,0.0,0.0
155832,311666,0.0,0.0,0.0
155833,311668,-0.0,-0.0,-0.0


In [74]:
sub=pd.melt(a, id_vars=['Id'], value_vars=['q0.05','q0.5','q0.95'])
sub['variable']=sub['variable'].str.replace("q","", regex=False)
sub['ForecastId_Quantile']=sub['Id'].astype(str)+'_'+sub['variable']
sub['TargetValue']=sub['value']
sub=sub[['ForecastId_Quantile','TargetValue']]
sub.reset_index(drop=True,inplace=True)
sub1 = sub.copy()
sub1

,ForecastId_Quantile,TargetValue
0,1_0.05,161.0
1,3_0.05,127.0
2,5_0.05,122.0
3,7_0.05,223.0
4,9_0.05,164.0
...,...,...
467500,311661_0.95,15.0
467501,311663_0.95,3.0
467502,311665_0.95,9.0
467503,311667_0.95,25.0


In [75]:
sub=pd.melt(a2, id_vars=['Id'], value_vars=['q0.05','q0.5','q0.95'])
sub['variable']=sub['variable'].str.replace("q","", regex=False)
sub['ForecastId_Quantile']=sub['Id'].astype(str)+'_'+sub['variable']
sub['TargetValue']=sub['value']
sub=sub[['ForecastId_Quantile','TargetValue']]
sub.reset_index(drop=True,inplace=True)
sub2 = sub.copy()
sub2

,ForecastId_Quantile,TargetValue
0,2_0.05,7.0
1,4_0.05,1.0
2,6_0.05,2.0
3,8_0.05,4.0
4,10_0.05,4.0
...,...,...
467500,311662_0.95,0.0
467501,311664_0.95,0.0
467502,311666_0.95,0.0
467503,311668_0.95,-0.0


In [76]:
sub = sub1.append(sub2)
sub

,ForecastId_Quantile,TargetValue
0,1_0.05,161.0
1,3_0.05,127.0
2,5_0.05,122.0
3,7_0.05,223.0
4,9_0.05,164.0
...,...,...
467500,311662_0.95,0.0
467501,311664_0.95,0.0
467502,311666_0.95,0.0
467503,311668_0.95,-0.0


In [77]:
sub.reset_index(drop=True,inplace=True)
sub.to_csv("/content/drive/MyDrive/Colab Notebooks/Проект/submission.csv",index=False)
sub.head()

,ForecastId_Quantile,TargetValue
0,1_0.05,161.0
1,3_0.05,127.0
2,5_0.05,122.0
3,7_0.05,223.0
4,9_0.05,164.0


In [78]:
sub.tail()

,ForecastId_Quantile,TargetValue
935005,311662_0.95,0.0
935006,311664_0.95,0.0
935007,311666_0.95,0.0
935008,311668_0.95,-0.0
935009,311670_0.95,-0.0


Выводы: Использование XGBoosting'a с параметрами (n_estimators = 2300 , alpha = 0, gamma = 0, learning_rate = 0.03,  random_state = 42 , max_depth = 23) дает нам наилучушую модель. Несмотря на всё проведенное исследоавние, мы можем сделать вывод о том, что предсказывать пандемию очень сложно. Это связано с тем, что мы работаем только со случаем смертей и заболеваний, модель не может выделить понятие волны или вспышки заболевания, она не учитывает дату (выходные - больше заболеваний - приходятся на дату, когда сделан тест). Несмотря на все минусы, средняя квадратическая ошибка является адекватным значением, что говорит нам о том, что наша модель не переобучена. Однако по оптимальным значениям, если сравнивать с официальными данными, то модель недозаражает и недоубивает людей. Это связано с такой концепцией, как график пандемии, который изачально растёт экспоненциально, а затем логарифмически. https://prnt.sc/w9w352
Таким образом, можно дополнить этот проект потенциальными данными по некоторым предыдущим эпидемиям, более ранним штаммам ковида, h1n1, etc. Продолжение следует
